# Cargo dependency breakdown

This notebook explores the relationship between dependencies and download counts in a bunch of Cargo.lock files found in the wild, in a crates.io database dump.

In [ ]:
import plotly.io as pio
print(pio.renderers.default)

In [ ]:
import plotly.express

In [ ]:
import pandas

In [ ]:
import psycopg2


In [ ]:
# This is a breakdown of dependency sub-trees found in Cargo.lock files in the wild.
trees = pandas.read_parquet("../subtrees-clean.parquet")

tree_counts = trees.groupby(
    ['package_name', 'package_version', 'hash'],
).agg(
    tree_size=('deps_count', 'first'),
    example_repo_path=('repo_path', 'first'),
    tree_occurrences=('repo_path', 'count'),
)

version_counts = tree_counts.groupby(
    ['package_name', 'package_version'],
).agg(
    version_occurrences=('tree_occurrences', 'sum'),
)


In [ ]:
def call_cached(fn):
    """
    This is a little cacheing pseudo-decorator, that I use to make things a bit
    quicker when re-running the notebook from scratch.
    It also means that you don't need to be running a postgresql server with the
    crates.io dump as long as you don't change any of the sql queries.
    """
    import inspect
    import hashlib
    digest = hashlib.sha256(inspect.getsource(get_downloads_data).encode()).hexdigest()
    cache_filename = f"../cache/{fn.__name__}-{digest}.parquet"

    try:
        data = pandas.read_parquet(cache_filename)
        return data
    except FileNotFoundError:
        pass

    data = fn()
    data.to_parquet(cache_filename)
    # TODO:
    # * Delete every file matching f"../cache/{fn.__name__}-*.parquet"
    #   other than `cache_filename`.
    #   (or touch the file we used, if we want an LRU cache with size bigger than 1)
    # * Log cache misses and timings.
    return data

In [ ]:
def get_downloads_data():
    """
    We care about how often dependency subtrees appear in our public Cargo.lock dataset
    (to see how fragmented the configurations for each package are),
    but we also want to scale by how many times the various packages get downloaded,
    to give us the overall weight for each crate. This function gives us these numbers.
    """
    conn = psycopg2.connect(
        database="cratesio",
    )
    downloads = pandas.read_sql_query("""
        select
            c.name as package_name, v.num as package_version, d.downloads
        from
            version_downloads as d
        join
            versions as v on v.id = d.version_id
        join
            crates as c on c.id = v.crate_id
        where
            date = '2021-03-29'
        order by
            package_name, package_version
        ;
    """, conn)
    return downloads

downloads = call_cached(get_downloads_data).set_index(['package_name', 'package_version'])

downloads

In [ ]:
combined = combined = tree_counts.join(
    version_counts,
    how='inner',
    on=['package_name', 'package_version']
).join(
    downloads,
    how='inner',
    on=['package_name', 'package_version'],
)
combined['estimated_daily_downloads'] = (
    combined['downloads'] * combined['tree_occurrences'] / combined['version_occurrences']
)


In [ ]:
def plot_unscaled(df, *, x, y, hover_name='package_name', color=None):
    plot_data = df.reset_index().drop_duplicates(subset=[x, y])
    fig = plotly.express.scatter(
        plot_data, x=x, y=y, color=color,
        hover_name=hover_name,
    )
    if hover_name is None:
        fig.update_traces(hovertemplate=None, hoverinfo='skip')
    return fig

plot_unscaled(combined, x='tree_occurrences', y='tree_size', hover_name=None)

# tree_size is the number of crates in the dependency subtree
# tree_occurrences is a count of how many Cargo.lock files a particular dependency sub-tree exists in
# This plot isn't super-useful. Skip to the next one for better axes.

In [ ]:
def plot_loglog(df, *, x, y, hover_name='package_name', color=None):
    plot_data = df.reset_index().drop_duplicates(subset=[x, y])
    fig = plotly.express.scatter(
        plot_data, x=x, y=y, hover_name=hover_name, color=color,
        log_x=True, log_y=True, 
    )
    # fig.update_traces(hovertemplate=None, hoverinfo='skip')
    return fig

plot_loglog(combined, x='tree_occurrences', y='tree_size')


* `tree_size` is the number of crates in the dependency subtree
* `tree_occurrences` is a count of how many Cargo.lock files a particular dependency sub-tree exists in
* Things out in the bottom-right are leaf-dependencies that are depended upon by loads of people
* Things in the top-left are huge dependency trees that only show up in one Cargo.lock file
  (note that there are a bunch of copies of amethyst and actix-web, with subtly different versions
  of their transitive sub-dependencies).

Note that the x axis is not scaled by how many times I expect particular trees to be built - just by how many public repos contain the tree.


PNG version of the figure is copied below for people viewing on GitHub. The real thing has the names of tree-root crates on hover.
![](./tree_size-vs-tree_occurrences.png)

In [ ]:
plot_loglog(combined, x='estimated_daily_downloads', y='tree_size')

This is the same graph, but scaled by how many times each crate is downloaded on crates.io. For each crate-version, we count how many times it appears at a dependency sub-tree root, and divide its crates.io download count by this number.

PNG version of the figure is copied below for people viewing on GitHub. The real thing has the names of tree-root crates on hover.

![](tree_size-vs-esitmated_daily_downloads.png)

## Analysis

the utility gained by building a package tree is proportional to the number of users (tree_ocurrences) * the size of the tree (tree_size).

    utility = tree_ocurrences * tree_size
    log(utility) = log(tree_ocurrences * tree_size)
    log(utility) = log(tree_ocurrences) + log(tree_size)

therefore, lines of constant utility are straight downwards-sloping lines in the above log plot (we want to build the packages that are furthest to the top-right on this plot, like `frame-benchmarking`, `mio` and `winapi`)

In practice, tree_ocurrences only says how many *GitHub Repos* are using particular configurations of the crates. We want to know how many *people* are using them. For this, we need to use the crates.io download data.

## Caveats

The set of requested features do not seem to appear in the lockfile. They only appear in Cargo.toml. In a lot of cases, changing features will add extra dependencies, but not always. We may need to re-do the analysis and create a new `subtrees-clean.parquet` that adds a "features" field to each dependency in the tree. This could be a lot of work with not great payoff though. Might be best to just keep in mind that the fragmentation will be a bit worse than what you see on these graphs.

I am assuming that build time is proportional to dependency tree size. In reality, it is likely to also scale proportional to crate size (available as `versions.crate_size` in the crates.io postgresql dump), and whether it is using a lot of proc-macros and generics from its dependencies. Predicting crate build times would be a really interesting project, if anyone has a dataset (maybe crater has one?).

# Draw a line at 20% build-time-saved

How many packages do we need to build in order to save 20% of people's time?

Assuming:
* Each crate is compiled once immediately after download and then never again (not true, but the repeated compiles probably scale with the number of downloads, so I'm hoping it falls out in the wash)
* !!! Each crate takes a time proportional to its dependency count to compile (!!! I don't think that this is valid, because it assumes that there will be build time incurred by leaf dependencies at each layer above, even though the leaf dep's build time has already been counted based on the download count of the leaf dependency)
* !!! It is possible to build a crate without building its leaf-dependencies (!!! In practice, in order to build a level-above crate, you need to reach into the quagmire and build the leaf-dependencies, even if we haven't identified them as good things to build)

Let's just get a graph drawn and go from there, shall we?

In [ ]:
# fig = plot_loglog(combined, x='estimated_daily_downloads', y='tree_size')

combined['time_wasted_compiling'] = combined['estimated_daily_downloads'] * combined['tree_size']
time_wasted_compiling = combined['time_wasted_compiling']

In [ ]:
plot_loglog(combined, x='estimated_daily_downloads', y='tree_size', color='time_wasted_compiling')

PNG included below for people viewing on GitHub.

![](./time_wasted_compiling.png)

In [ ]:
time_wasted_compiling.describe()

In [ ]:
time_wasted_compiling = time_wasted_compiling.sort_values(ascending=False)

In [ ]:
cumulative_cost = time_wasted_compiling.cumsum()

In [ ]:
cumulative_cost[-1] / 4

In [ ]:
combined['in_25_percent'] = cumulative_cost < (cumulative_cost[-1] / 4)

In [ ]:
plot_loglog(combined, x='estimated_daily_downloads', y='tree_size', color='in_25_percent')

In [ ]:
combined['in_25_percent'].value_counts()

What if we give ourselves a goal of saving 25% of all computation done by `cargo build` invocations?

PNG for GitHub users:
![](./in_25_percent.png)

we would need to build 1728 package trees, and can skip 574515 trees.

# But what if we only care about downloads?

Editor's note: I can't remember why I thought that this was a valid simplifying assumption. It may not actually be valid.

In [ ]:
cumulative_daily_downloads = combined['estimated_daily_downloads'].sort_values(ascending=False).cumsum()
combined['cumulative_daily_downloads'] = cumulative_daily_downloads
in_25_percent_downloads = cumulative_daily_downloads < (cumulative_daily_downloads[-1] / 4)

In [ ]:
in_25_percent_downloads.value_counts()

In [ ]:
combined['in_25_percent_downloads'] = in_25_percent_downloads

In [ ]:
plot_unscaled(combined, x='estimated_daily_downloads', y='cumulative_daily_downloads', color='in_25_percent_downloads')

In this case, we would only need to build 116 packages for 25% coverage. I'm not sure how valid this is though.

![](./downloads_25.png)

In [ ]:
combined[in_25_percent_downloads]

In [ ]:
combined[in_25_percent_downloads].to_csv('../top_25_percent_downloads.csv')

Things that could screw me over:

* If a package version shows up in the crates.io download counts, but not in the github scraped lockfiles, I ignore it completely.
* I still don't take into account features that don't affect the shape of the dependency tree. 
* I've not had very good sleep for 2 days straight, so I've probably made a mistake somewhere